In [1]:
import numpy as np

In [2]:
# Return true if a and b form a base pair
def is_base_pair(a, b):
    if a == 'A':
        return b == 'U'
    if a == 'U':
        return b == 'A'
    if a == 'C':
        return b == 'G'
    if a == 'G':
        return b == 'C'
    return False

# Given RNA sequence v, return an n x n matrix S
# where S[i, j] is the maximum number of
# pseudoknot-free base pairs in subsequence vi through vj
def nussinov(sequence):
    n = len(sequence)
    dp_matrix = np.zeros((n,n)) # The n x n matrix S for dynamic programming

    for k in range(n - 1): # There are k = n - 1 iterations
        for i in range(n - k - 1): # There are i = n - k - 1 entries to compute in kth iteration 
            j = i + k + 1

            # Case 1: i and j form a base pair
            case_1 = 0
            if is_base_pair(sequence[i], sequence[j]):
                case_1 = dp_matrix[i + 1, j - 1] + 1

            # Case 2: i is unpaired
            case_2 = dp_matrix[i + 1, j]

            # Case 3: j is unpaired
            case_3 = dp_matrix[i, j - 1]

            # Case 4: bifurcation
            case_4 = 0
            for b in range(i + 1, j):
                curr_value = dp_matrix[i, b] + dp_matrix[b + 1, j]
                if curr_value > case_4:
                    case_4 = curr_value

            # Take the maximum of all four cases above
            dp_matrix[i, j] = max(case_1, case_2, case_3, case_4)

    return dp_matrix

# Given the n x n matrix S from Nussinov Algorithm, 
# return the secondary structure as a list of index pairs
def traceback(dp_matrix):
    n = dp_matrix.shape[0]
    stack = [] # a stack for the traceback step
    base_pairs = [] # the list of index pairs

    stack.append((0, n - 1))
    while stack:
        i, j = stack.pop()

        # Base case
        if i >= j:
            continue

        # Case 2: i is unpaired
        if dp_matrix[i + 1, j] == dp_matrix[i, j]:
            stack.append((i + 1, j))

        # Case 3: j is unpaired
        elif dp_matrix[i, j - 1] == dp_matrix[i, j]:
            stack.append((i, j - 1))

        # Case 1: i and j form a base pair
        elif dp_matrix[i + 1, j - 1] + 1 == dp_matrix[i, j]:
            base_pairs.append((i, j))
            stack.append((i + 1, j - 1))

        # Case 4: bifurcation
        else:
            for b in range(i + 1, j):
                if dp_matrix[i, b] + dp_matrix[b + 1, j] == dp_matrix[i, j]:
                    stack.append((b + 1, j))
                    stack.append((i, b))
                    break

    return base_pairs

In [3]:
# Given RNA sequence v, return an n x n matrix S
# and backpointers which prioritize case 1
def nussinov_with_backpointers(sequence):
    n = len(sequence)
    dp_matrix = np.zeros((n,n)) # The n x n matrix S for dynamic programming
    backpointers = np.zeros((n,n)) # The n x n matrix of backpointers

    for k in range(n - 1): # There are k = n - 1 iterations
        for i in range(n - k - 1): # There are i = n - k - 1 entries to compute in kth iteration 
            j = i + k + 1

            # Case 1: i and j form a base pair
            case_1 = 0
            if is_base_pair(sequence[i], sequence[j]):
                case_1 = dp_matrix[i + 1, j - 1] + 1

            # Case 2: i is unpaired
            case_2 = dp_matrix[i + 1, j]

            # Case 3: j is unpaired
            case_3 = dp_matrix[i, j - 1]

            # Case 4: bifurcation
            case_4 = 0
            for b in range(i + 1, j):
                curr_value = dp_matrix[i, b] + dp_matrix[b + 1, j]
                if curr_value > case_4:
                    case_4 = curr_value

            # Take the maximum of all four cases above
            dp_matrix[i, j] = max(case_1, case_2, case_3, case_4)

            # Set the corresponding backpointer
            if dp_matrix[i, j] == 0:
                backpointers[i, j] = 2
            elif dp_matrix[i, j] == case_1:
                backpointers[i, j] = 1
            elif dp_matrix[i, j] == case_2:
                backpointers[i, j] = 2
            elif dp_matrix[i, j] == case_3:
                backpointers[i, j] = 3
            else:
                backpointers[i, j] = 4

    return dp_matrix, backpointers

# Given the n x n matrix S and backpointers from Nussinov Algorithm, 
# return the secondary structure as a list of index pairs
def traceback_with_backpointers(dp_matrix, backpointers):
    n = dp_matrix.shape[0]
    stack = [] # a stack for the traceback step
    base_pairs = [] # the list of index pairs

    stack.append((0, n - 1))
    while stack:
        i, j = stack.pop()

        # Base case
        if i >= j:
            continue

        # Case 1: i and j form a base pair
        if backpointers[i, j] == 1:
            base_pairs.append((i, j))
            stack.append((i + 1, j - 1))

        # Case 2: i is unpaired
        elif backpointers[i, j] == 2:
            stack.append((i + 1, j))

        # Case 3: j is unpaired
        elif backpointers[i, j] == 3:
            stack.append((i, j - 1))

        # Case 4: bifurcation
        else:
            for b in range(i + 1, j):
                if dp_matrix[i, b] + dp_matrix[b + 1, j] == dp_matrix[i, j]:
                    stack.append((b + 1, j))
                    stack.append((i, b))
                    break

    return base_pairs

In [4]:
# Return the number of same base pairs in a and b
def compare_base_pairs(a, b):
    count = 0
    for base_pair in a:
        if base_pair in b:
            count += 1
    return count

# Given the sequence and index pairs, print the predicted base pairs
def print_base_pairs(sequence, base_pairs):
    for i, j in base_pairs:
        print((sequence[i], sequence[j]))

In [5]:
# Test correctness on mock sequences
mock_1 = 'GGGAAAUCC'

dp_matrix = nussinov(mock_1)
base_pairs = traceback(dp_matrix)
print('The score by Nussinov Algorithm: ' + str(len(base_pairs)))
print('The base pairs: ' + str(base_pairs))
print_base_pairs(mock_1, base_pairs)
print('')

dp_matrix_bp, backpointers = nussinov_with_backpointers(mock_1)
base_pairs_bp = traceback_with_backpointers(dp_matrix_bp, backpointers)
print('The score by Nussinov Algorithm with backpointers: ' + str(len(base_pairs_bp)))
print('The base pairs: ' + str(base_pairs_bp))
print_base_pairs(mock_1, base_pairs_bp)

The score by Nussinov Algorithm: 3
The base pairs: [(1, 8), (2, 7), (5, 6)]
('G', 'C')
('G', 'C')
('A', 'U')

The score by Nussinov Algorithm with backpointers: 3
The base pairs: [(0, 8), (1, 7), (3, 6)]
('G', 'C')
('G', 'C')
('A', 'U')


In [6]:
mock_2 = 'GCUCGGGUUCCCUAUUCAAGAGC'

dp_matrix = nussinov(mock_2)
base_pairs = traceback(dp_matrix)
print('The score by Nussinov Algorithm: ' + str(len(base_pairs)))
print('The base pairs: ' + str(base_pairs))
print_base_pairs(mock_2, base_pairs)
print('')

dp_matrix_bp, backpointers = nussinov_with_backpointers(mock_2)
base_pairs_bp = traceback_with_backpointers(dp_matrix_bp, backpointers)
print('The score by Nussinov Algorithm with backpointers: ' + str(len(base_pairs_bp)))
print('The base pairs: ' + str(base_pairs_bp))
print_base_pairs(mock_2, base_pairs_bp)

The score by Nussinov Algorithm: 10
The base pairs: [(0, 22), (1, 21), (2, 20), (3, 19), (4, 11), (5, 10), (6, 9), (12, 18), (13, 14), (15, 17)]
('G', 'C')
('C', 'G')
('U', 'A')
('C', 'G')
('G', 'C')
('G', 'C')
('G', 'C')
('U', 'A')
('A', 'U')
('U', 'A')

The score by Nussinov Algorithm with backpointers: 10
The base pairs: [(0, 22), (1, 21), (2, 20), (3, 19), (4, 11), (5, 10), (6, 9), (12, 18), (13, 14), (15, 17)]
('G', 'C')
('C', 'G')
('U', 'A')
('C', 'G')
('G', 'C')
('G', 'C')
('G', 'C')
('U', 'A')
('A', 'U')
('U', 'A')


In [7]:
# Note the incorrect A-C pair predicted by Nussinov without backpointers
mock_3 = 'UUUCCUGAAACCUCGGCAACAAUUGAA'

dp_matrix = nussinov(mock_3)
base_pairs = traceback(dp_matrix)
print('The score by Nussinov Algorithm: ' + str(len(base_pairs)))
print('The base pairs: ' + str(base_pairs))
print_base_pairs(mock_3, base_pairs)
print('')

dp_matrix_bp, backpointers = nussinov_with_backpointers(mock_3)
base_pairs_bp = traceback_with_backpointers(dp_matrix_bp, backpointers)
print('The score by Nussinov Algorithm with backpointers: ' + str(len(base_pairs_bp)))
print('The base pairs: ' + str(base_pairs_bp))
print_base_pairs(mock_3, base_pairs_bp)

The score by Nussinov Algorithm: 10
The base pairs: [(1, 26), (2, 25), (5, 18), (6, 17), (9, 16), (11, 15), (13, 14), (19, 24), (20, 23), (21, 22)]
('U', 'A')
('U', 'A')
('U', 'A')
('G', 'A')
('A', 'C')
('C', 'G')
('C', 'G')
('C', 'G')
('A', 'U')
('A', 'U')

The score by Nussinov Algorithm with backpointers: 10
The base pairs: [(0, 26), (1, 25), (3, 24), (5, 18), (6, 10), (12, 17), (13, 14), (15, 16), (20, 23), (21, 22)]
('U', 'A')
('U', 'A')
('C', 'G')
('U', 'A')
('G', 'C')
('U', 'A')
('C', 'G')
('G', 'C')
('A', 'U')
('A', 'U')


In [8]:
# Test performance on real sequences from Rfam
file_1 = open('RF00162_AF027868.1_5245-5154.fasta', 'r')
sequence_1 = file_1.read()
file_1.close()
idx = sequence_1.find('\n')
sequence_1 = sequence_1[idx + 1:]

# The alignment by Rfam
alignment_1 = [(0, 91), (1, 90), (3, 88), (4, 87), (5, 86), (6, 85), (7, 84), (14, 39), (15, 38), (16, 37), 
               (20, 30), (21, 29), (22, 28), (42, 58), (43, 57), (44, 56), (46, 55), (47, 54), (48, 53), 
               (67, 80), (68, 79), (69, 78), (70, 77)]

In [9]:
print('The sequence: ' + sequence_1)
print('The score by Rfam: ' + str(len(alignment_1)) + '\n')

dp_matrix = nussinov(sequence_1)
base_pairs = traceback(dp_matrix)
print('The score by Nussinov Algorithm: ' + str(len(base_pairs)))
print('Similarity: ' + str(compare_base_pairs(base_pairs, alignment_1)) + '\n')

dp_matrix_bp, backpointers = nussinov_with_backpointers(sequence_1)
base_pairs_bp = traceback_with_backpointers(dp_matrix_bp, backpointers)
print('The score by Nussinov Algorithm with backpointers: ' + str(len(base_pairs_bp)))
print('Similarity: ' + str(compare_base_pairs(base_pairs_bp, alignment_1)))

The sequence: UUUCUAUCCAGAGAGGUGGAGGGACUGGCCCUAUGAAACCUCGGCAACAUUAUUGUGCCAAUUCCAGCAAGCGCUAGCUUGAAAGAUAGGAA
The score by Rfam: 23

The score by Nussinov Algorithm: 39
Similarity: 7

The score by Nussinov Algorithm with backpointers: 39
Similarity: 3


In [10]:
file_2 = open('RF03135_URS0001A2386A_28892.fasta', 'r')
sequence_2 = file_2.read()
file_2.close()
idx = sequence_2.find('\n')
sequence_2 = sequence_2[idx + 1:-1]

# The alignment by Rfam
alignment_2 = [(0, 72), (1, 71), (2, 70), (3, 69), (4, 68), (7, 63), (8, 62), (9, 61), (14, 39), 
               (15, 38), (16, 37), (17, 36), (21, 34), (22, 33), (23, 32), (24, 31), (42, 56), (43, 55), 
               (44, 54), (45, 53), (46, 52)]

In [11]:
print('The sequence: ' + sequence_2)
print('The score by Rfam: ' + str(len(alignment_2)) + '\n')

dp_matrix = nussinov(sequence_2)
base_pairs = traceback(dp_matrix)
print('The score by Nussinov Algorithm: ' + str(len(base_pairs)))
print('Similarity: ' + str(compare_base_pairs(base_pairs, alignment_2)) + '\n')

dp_matrix_bp, backpointers = nussinov_with_backpointers(sequence_2)
base_pairs_bp = traceback_with_backpointers(dp_matrix_bp, backpointers)
print('The score by Nussinov Algorithm with backpointers: ' + str(len(base_pairs_bp)))
print('Similarity: ' + str(compare_base_pairs(base_pairs_bp, alignment_2)))

The sequence: CUGAUCCGUGUAAUGGGCCUCCGCCCGUGCAGGCGAGCCCGGCCGGCCCCGUGCCGGCAUGCACAACGAUCAG
The score by Rfam: 21

The score by Nussinov Algorithm: 34
Similarity: 4

The score by Nussinov Algorithm with backpointers: 34
Similarity: 8
